In [11]:
using DataFrames,PythonPlot,StatsBase,Random,Distributions,CSV,DSP,LinearAlgebra
PythonPlot.svg(true)
include("model.jl")
include("process_mm_data.jl")

FIG_PATH = "/Users/elevien/Dartmouth College Dropbox/Ethan Levien/RESEARCH/ACTIVE/cell-size/linear_model/figures"

"/Users/elevien/Dartmouth College Dropbox/Ethan Levien/RESEARCH/ACTIVE/cell-size/linear_model/figures"

In [12]:
# make dataframe from CSV file
DATA_PATH = "./data/MC4100_25C"
data_mm = [CSV.read(DATA_PATH*"/"*f,DataFrame,header=string.(["row","div","length","x","z"])) for f in readdir(DATA_PATH)]
data_mm = vcat([hcat(data_mm[i],DataFrame(:lineage=>i*ones(length(data_mm[i][:,1])))) for i in eachindex(data_mm)]...)
data_mm = combine(groupby(data_mm,:lineage),:div => cumsum => :n,names(data_mm));
data_mm = process_lineage(data_mm)

   # Generate cell-level statistics
dfcell_mm = combine(groupby(data_mm,[:lineage,:cell]),
   :z0 => (x -> x[1]) => :z0,
   :time => (x -> x[end]-x[1]) => :τ,
   :Y => (x -> x[end] - x[1]) => :ϕ
);

dfcell_mm.λ = dfcell_mm.ϕ ./ dfcell_mm.τ
dfcell_mm.τ = dfcell_mm.τ ./  mean(dfcell_mm.τ);

In [13]:
dfcell_mm

Row,lineage,cell,z0,τ,ϕ,λ
,Float64,Int64,Float64,Float64,Float64,Float64
1,1.0,1,0.422715,1.06459,0.859798,0.0121098
2,1.0,2,0.561414,0.914651,0.705071,0.0115585
3,1.0,3,0.323003,1.09458,0.944634,0.0129402
4,1.0,4,0.517173,1.03461,0.785685,0.0113867
5,1.0,5,0.470022,1.12457,0.900384,0.0120051
6,1.0,6,0.544084,1.15456,1.02944,0.0133694
7,1.0,7,0.589685,1.06459,0.708207,0.00997475
8,1.0,8,0.53992,1.0496,0.813016,0.0116145
9,1.0,9,0.600686,0.854674,0.685105,0.0120194
